## Model

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from neural_verification import GeneralRNNConfig, GeneralRNN

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
model_config = GeneralRNNConfig(
        input_dim=1,
        output_dim=1,
        hidden_dim=2,
        hidden_mlp_depth=2,
        hidden_mlp_width=3,
        output_mlp_depth=1,
        output_mlp_width=1,
        activation=nn.ReLU
        )

model = GeneralRNN(model_config, device=device)

In [4]:
model

GeneralRNN(
  (hmlp): MLP(
    (mlp): Sequential(
      (0): Linear(in_features=3, out_features=3, bias=True)
      (1): ReLU()
      (2): Linear(in_features=3, out_features=2, bias=True)
    )
  )
  (ymlp): MLP(
    (mlp): Sequential(
      (0): Linear(in_features=2, out_features=1, bias=True)
    )
  )
)

In [5]:
model.hmlp.mlp[0].weight.data = torch.tensor([[ 1.,  0.,  1.],
                                              [-1.,  0., -1.],
                                              [ 0.,  1.,  0.]])

In [6]:
model.hmlp.mlp[0].bias.data = torch.tensor([0., 0., 0.])

In [7]:
model.hmlp.mlp[2].weight.data = torch.tensor([[ 1.,  -1.,  0.],
                                              [ 0.,  -1.,  1.]])

In [8]:
model.hmlp.mlp[2].bias.data = torch.tensor([0., 0.])

In [9]:
model.ymlp.mlp[0].weight.data = torch.tensor([[ 1.,  0.],
                                              [ 0., -1.]])

In [10]:
model.ymlp.mlp[0].bias.data = torch.tensor([0., 0.])

## Balanced Parentheses

In [34]:
def isBalanced(paren):
    iparen = []
    for p in paren:
        if p == "(":
            iparen.append(1.)
        elif p == ")":
            iparen.append(-1.)
    iparen = torch.tensor(iparen)
    iparen = torch.unsqueeze(iparen, 1)
    iparen = torch.unsqueeze(iparen, 0)
    
    outs, hiddens = model.forward_sequence(iparen)
    print(list(h[0].tolist() for h in hiddens))
    return not outs.tolist()[0][-1][0]

## Testing

In [35]:
isBalanced("()()()()")

[[0.0, 0.0], [1.0, 0.0], [0.0, 0.0], [1.0, 0.0], [0.0, 0.0], [1.0, 0.0], [0.0, 0.0], [1.0, 0.0], [0.0, 0.0]]


True

In [36]:
isBalanced("()")

[[0.0, 0.0], [1.0, 0.0], [0.0, 0.0]]


True

In [37]:
isBalanced("(())")

[[0.0, 0.0], [1.0, 0.0], [2.0, 0.0], [1.0, 0.0], [0.0, 0.0]]


True

In [38]:
isBalanced("(())()")

[[0.0, 0.0], [1.0, 0.0], [2.0, 0.0], [1.0, 0.0], [0.0, 0.0], [1.0, 0.0], [0.0, 0.0]]


True

In [39]:
isBalanced("(()())()()")

[[0.0, 0.0], [1.0, 0.0], [2.0, 0.0], [1.0, 0.0], [2.0, 0.0], [1.0, 0.0], [0.0, 0.0], [1.0, 0.0], [0.0, 0.0], [1.0, 0.0], [0.0, 0.0]]


True

In [40]:
isBalanced("(()(())()())")

[[0.0, 0.0], [1.0, 0.0], [2.0, 0.0], [1.0, 0.0], [2.0, 0.0], [3.0, 0.0], [2.0, 0.0], [1.0, 0.0], [2.0, 0.0], [1.0, 0.0], [2.0, 0.0], [1.0, 0.0], [0.0, 0.0]]


True

In [41]:
isBalanced("(")

[[0.0, 0.0], [1.0, 0.0]]


False

In [42]:
isBalanced(")")

[[0.0, 0.0], [-1.0, -1.0]]


False

In [43]:
isBalanced("(()")

[[0.0, 0.0], [1.0, 0.0], [2.0, 0.0], [1.0, 0.0]]


False

In [44]:
isBalanced("()()(())())(")

[[0.0, 0.0], [1.0, 0.0], [0.0, 0.0], [1.0, 0.0], [0.0, 0.0], [1.0, 0.0], [2.0, 0.0], [1.0, 0.0], [0.0, 0.0], [1.0, 0.0], [0.0, 0.0], [-1.0, -1.0], [0.0, -1.0]]


False

In [45]:
isBalanced(")(")

[[0.0, 0.0], [-1.0, -1.0], [0.0, -1.0]]


False

In [47]:
isBalanced("()((())(()((()())())(())(((()()())())())))))")

[[0.0, 0.0], [1.0, 0.0], [0.0, 0.0], [1.0, 0.0], [2.0, 0.0], [3.0, 0.0], [2.0, 0.0], [1.0, 0.0], [2.0, 0.0], [3.0, 0.0], [2.0, 0.0], [3.0, 0.0], [4.0, 0.0], [5.0, 0.0], [4.0, 0.0], [5.0, 0.0], [4.0, 0.0], [3.0, 0.0], [4.0, 0.0], [3.0, 0.0], [2.0, 0.0], [3.0, 0.0], [4.0, 0.0], [3.0, 0.0], [2.0, 0.0], [3.0, 0.0], [4.0, 0.0], [5.0, 0.0], [6.0, 0.0], [5.0, 0.0], [6.0, 0.0], [5.0, 0.0], [6.0, 0.0], [5.0, 0.0], [4.0, 0.0], [5.0, 0.0], [4.0, 0.0], [3.0, 0.0], [4.0, 0.0], [3.0, 0.0], [2.0, 0.0], [1.0, 0.0], [0.0, 0.0], [-1.0, -1.0], [-2.0, -3.0]]


False